In [1]:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall19/week03_convnets/cifar.py

--2020-02-09 08:58:05--  https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall19/week03_convnets/cifar.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.244.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2396 (2.3K) [text/plain]
Saving to: ‘cifar.py’

cifar.py            100%[===================>]   2.34K  --.-KB/s    in 0s      

2020-02-09 08:58:06 (31.0 MB/s) - ‘cifar.py’ saved [2396/2396]



In [1]:
import numpy as np
from cifar import load_cifar10
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10("cifar_data")

class_names = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer',
                        'dog', 'frog', 'horse', 'ship',
                        'truck'])

print(X_train.shape,y_train.shape)

(40000, 3, 32, 32) (40000,)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=[12,10])
for i in range(12):
    plt.subplot(3,4,i+1)
    plt.xlabel(class_names[y_train[i]])
    plt.imshow(np.transpose(X_train[i],[1,2,0]))

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Переведем данные в торч тензоры.

In [5]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.int64)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)
X_train.shape

torch.Size([40000, 3, 32, 32])

In [6]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# Создайте модель

In [7]:
model = nn.Sequential()
model.add_module('conv1', nn.Conv2d(

# Определите оптимизатор

In [8]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
compute_loss =  nn.NLLLoss()

In [9]:
history = []
N_ITERS = 3000
BATCH_SIZE = 64

In [ ]:
model.train(True)

for i in range(N_ITERS):
    idx = np.random.choice(np.arange(0, X_train.shape[0]), size=BATCH_SIZE, replace=False)
    x_batch = X_train[idx]
    y_batch = y_train[idx]
    y_pred = model(x_batch)
    loss = compute_loss(y_pred, y_batch)
    loss.backward()
    opt.step()
    
    opt.zero_grad()
    history.append(loss)

 # Точность на валидации

In [20]:
model.train(False)

test_batch_acc = []
for i in range(0, len(X_val), 100):
    x_batch = X_val[i: i+100]
    y_batch = y_val[i: i+100].data.numpy()
    logits = model(x_batch)
    y_pred = logits.max(1)[1].data.numpy()
    test_batch_acc.append(np.mean(y_batch == y_pred))
    
print('Accuracy on validation {}'.format(np.mean(test_batch_acc)))

Accuracy on validation 0.7081999999999999


# Точность на тесте

In [21]:
model.train(False)

test_batch_acc = []
for i in range(0, len(X_test), 100):
    x_batch = X_test[i: i+100]
    y_batch = y_test[i: i+100].data.numpy()
    logits = model(x_batch)
    y_pred = logits.max(1)[1].data.numpy()
    test_batch_acc.append(np.mean(y_batch == y_pred))
    
print('Accuracy on test {}'.format(np.mean(test_batch_acc)))